# 복소수를 가중치로 가지는 복소 완전연결층

In [73]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

from keras import initializers 
igu = initializers.get('glorot_uniform')
iz = initializers.get('zeros')
  
class CDENSE(Layer):
    # FC: Simplified complex fully connected layer
    def __init__(self, No, **kwargs):
        self.No = No
        super().__init__(**kwargs)

    def build(self, inshape_l):
        inshape = inshape_l[0] # Real and Imag are the same shape
        self.w_r = self.add_weight("w_r", (inshape[1], self.No), initializer=igu)
        self.w_i = self.add_weight("w_i", (inshape[1], self.No), initializer=igu)
        self.b_r = self.add_weight("b_r", (self.No,), initializer=iz)        
        self.b_i = self.add_weight("b_i", (self.No,), initializer=iz) 
        self.w = tf.complex(self.w_r, self.w_i)
        self.b = tf.complex(self.b_r, self.b_i)
        super().build(inshape)  

    def call(self, x_l):
        x_r, x_i = x_l
        x = tf.complex(x_r, x_i)
        y = tf.matmul(x, self.w) + self.b
        y_r = tf.real(y)
        y_i = tf.imag(y)
        return [y_r, y_i]

    def compute_output_shape(self, inshape_l):
        # real and image are the same length
        return [(inshape_l[0], self.No), (inshape_l[0], self.No)]

    
def modeling(input_shape):
    x_r = keras.layers.Input(input_shape)
    x_i = keras.layers.Input(input_shape)
    [y_r, y_i] = CDENSE(1, input_shape=(1,))([x_r, x_i])
    return keras.models.Model([x_r, x_i], [y_r, y_i])


def cfit(model, x, y, **kwargs):
    x_l = [np.real(x), np.imag(x)]
    y_l = [np.real(y), np.imag(y)]        
    return model.fit(x_l, y_l, **kwargs)

def cpredict(model, x, **kwargs):
    x_l = [np.real(x), np.imag(x)]
    y_l = model.predict(x_l)
    return y_l[0] + 1j * y_l[1]

def cget_weights(model):
    [w_r, w_i, b_r, b_i] = model.get_weights()
    return([w_r + 1j * w_i], [b_r + 1j * b_i])    
    
def cmain():
    model = modeling((1,))
    model.compile(keras.optimizers.sgd(), 'mse')

    x = np.array([0, 1, 2, 3, 4]) + 1j*np.array([4, 3, 2, 1, 0])
    y = x * (2 + 1j) + (1 + 2j)

    # Training
    h = cfit(model, x[:2], y[:2], epochs=5000, verbose=0)
    
    # Testing
    y_pred = cpredict(model, x[2:])
    print('Targets:', y[2:])    
    print(y_pred) 

    # Weights
    [w, b] = cget_weights(model)
    print('weight:', w)
    print('bias:', b)

In [74]:
cmain()

Targets: [ 3.+8.j  6.+7.j  9.+6.j]
[[ 3.00877428+7.9657917j ]
 [ 6.01544905+6.94412136j]
 [ 9.02212429+5.92245197j]]
weight: [array([[ 2.01417232+0.99250239j]], dtype=complex64)]
bias: [array([ 0.96543461+1.95244229j], dtype=complex64)]


---
# KERASPP
## 코딩셰프의 3분 딥러닝, 케라스맛
[Keras 코드로 맛보는 ANN, DNN, CNN, RNN, AE, GAN, UNET](http://www.yes24.com/24/goods/57617933)

케라스 코드로 맛보는 딥러닝 핵심 개념! 

간결하고 직관적인 인공신경망 API를 제공하는 케라스는 구글 텐서플로, 마이크로소프트 CNTK, 아마존 MXNET, OpenCL PlaidML, 시애노 등의 딥러닝 엔진에서 지원하는 인기 인공지능 툴입니다. 이 코드들은 딥러닝 인공신경망 구현에 케라스를 사용합니다. 케라스로 주요 인공신경망인 ANN, DNN, CNN, RNN, AE, GAN, UNET을 구현하는 방법을 알아봅니다. 따라서 인공지능과 딥러닝 인공신경망의 구현에 관심이 있는 누구나 이 코드의 사용자입니다.

<p align="center">
<a href="http://www.yes24.com/24/goods/57617933">
<img src="http://image.yes24.com/momo/TopCate1742/MidCate007/174166602.jpg" alt="3분 딥러닝 케라스맛" title="3분 딥러닝 케라스맛" width="350"/>
</a>
</p>


## 구성

케라스를 이용해 딥러닝 인공신경망을 만들어 인공지능을 구현합니다. 1장은 케라스를 시작하는 데 필요한 기초를 다룹니다. 2장부터는 최신 인공지능 구현 방법인 주요 인공신경망을 예제로 이용해 다룹니다. 2장~5장에서 다루는 ANN, DNN, CNN, RNN은 지도학습 방식의 인공지능입니다. 6장과 7장에서 다루는 AE와 GAN은 비지도학습 방식이고 8장의 UNET은 고급 지도학습 방법입니다. 9장은 8장까지 배운 내용을 응용하고 확장하는 방법을 다룹니다. 

예제는 쉽게 인공지능 구현 방법을 익히고, 추후 실무에 쉽게 재사용할 수 있게 하는 데 주안점을 두어 작성했습니다. 